In [ ]:
import networkx as nx
import os
import json
import matplotlib.pyplot as plt

In [ ]:
G = nx.DiGraph()

data_directory = "./output_friends/"
my_screen_name="xalwudhal"

# Load the people I follow into a list
my_friends = []
with open(data_directory+ my_screen_name + ".json",encoding = 'utf-8') as f:
        friends=json.loads(f.read())
        my_friends = [friend["screen_name"] for friend in friends]

# There's something great about Twitter calling people these my friends.  I follow them, but there's no guarantee they actually *like* me.
my_friends[:10]

In [ ]:
# Load everyone's friends except mine as nodes in our graph
for filename in os.listdir(data_directory):
    if (filename != my_screen_name + ".json"):
        with open(data_directory+filename,encoding = 'utf-8') as f:
            screen_name = filename.replace(".json","")
            friends = json.loads(f.read())
            
            G.add_node(screen_name,is_friend=True)

            # Add all of my friend's friends
            for friend in friends:
                    # If this friend is in Bert's list of friends, assign the is_friend attribute appropriately
                    if friend["screen_name"] in my_friends:
                        is_friend = True
                    else:
                        is_friend = False

                    G.add_node(friend["screen_name"], is_friend = is_friend)
                    G.add_edge(screen_name,friend["screen_name"])
                

In [ ]:
# Find the people with the highest number of edges, filtering out the ones that are already my friends
degrees = [{"screen_name": n[0],  "degrees": list(nx.degree(G,n))[0][1]} for n in nx.get_node_attributes(G,'is_friend').items() if n[1] == False]
sorted_degrees = sorted(degrees, key=lambda k:k["degrees"], reverse=True)

# Get the top 50
top_50 = sorted_degrees[1:50]
top_50

In [ ]:
# Create a new graph that will only contain the top 50 friends of friends
G2 = nx.DiGraph()

data_directory = "./output_friends/"

# Load me into the graph
G2.add_node(my_screen_name, friend_tier=0)

# Add Bert's Friends
with open(data_directory+ my_screen_name + ".json",encoding = 'utf-8') as f:
        friends=json.loads(f.read())
        for friend in friends:
            G2.add_node(friend["screen_name"], friend_tier=1)
            G2.add_edge(my_screen_name, friend["screen_name"])

# Add Bert's Friends' Friends only if they appear in the top_50 list
for filename in os.listdir(data_directory):
    if (filename != my_screen_name + ".json"):
        with open(data_directory+filename,encoding = 'utf-8') as f:
            screen_name = filename.replace(".json","")
            friends = json.loads(f.read())

            for friend in friends:
                    # Add only the top_50 friends of friends
                    if any(d["screen_name"] == friend["screen_name"] for d in top_50):  
                        G2.add_node(friend["screen_name"], friend_tier=2)
                        G2.add_edge(screen_name,friend["screen_name"])

In [ ]:
# Shells are the circles that nodes appear on the graph
shell_level_1 = []
shell_level_2 = []

# Add nodes to the appropriate circle
for n in nx.get_node_attributes(G2,'friend_tier').items():
    if n[1] == 2:
        shell_level_2.append(n[0])
    else:
        shell_level_1.append(n[0])

shells = [shell_level_1,shell_level_2]
pos = nx.shell_layout(G2,shells,scale=500)
pos[my_screen_name] =[0,0]

nx.draw_networkx(G2,pos=pos,node_size=1,node_color="#cfcfcf",alpha=.3,arrowsize=4,font_size=4,width=.4,edge_color="#bdbdbd")
plt.savefig("Graph.png", dpi=1000)
plt.show()